In [1]:
#import library
import numpy as np
import pandas as pd
import yfinance as yf #Stock Data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings("ignore")
from transformers import pipeline
import finnhub #Company News
import torch

pd.set_option('display.max_colwidth',180)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-10-04 22:41:23.602296: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Companies top 500 

In [21]:
companies = pd.read_csv('/Users/ndridiby/Desktop/AI:ML Studio/S&P_500_companies.csv')
companies.head(10)

Symbol                Security             GICS Sector  \
0    MMM                      3M             Industrials   
1    AOS             A. O. Smith             Industrials   
2    ABT     Abbott Laboratories             Health Care   
3   ABBV                  AbbVie             Health Care   
4    ACN               Accenture  Information Technology   
5   ADBE              Adobe Inc.  Information Technology   
6    AMD  Advanced Micro Devices  Information Technology   
7    AES         AES Corporation               Utilities   
8    AFL                   Aflac              Financials   
9      A    Agilent Technologies             Health Care   

                              GICS Sub-Industry    Headquarters Location  \
0                      Industrial Conglomerates    Saint Paul, Minnesota   
1                             Building Products     Milwaukee, Wisconsin   
2                         Health Care Equipment  North Chicago, Illinois   
3                                 Biotechnology  North Chicago, Illinois   
4                IT Consulting & Other Services          Dublin, Ireland   
5                          Application Software     San Jose, California   
6                                Semiconductors  Santa Clara, California   
7  Independent Power Producers & Energy Traders      Arlington, Virginia   
8                       Life & Health Insurance        Columbus, Georgia   
9                Life Sciences Tools & Services  Santa Clara, California   

   Date added      CIK      Founded  
0  1957-03-04    66740         1902  
1  2017-07-26    91142         1916  
2  1957-03-04     1800         1888  
3  2012-12-31  1551152  2013 (1888)  
4  2011-07-06  1467373         1989  
5  1997-05-05   796343         1982  
6  2017-03-20     2488         1969  
7  1998-10-02   874761         1981  
8  1999-05-28     4977         1955  
9  2000-06-05  1090872         1999

## Stock Data

In [22]:
stock_data = yf.download('ABNB',start='2023-1-1',end=datetime.today())
stock_data = stock_data.reset_index()
stock_data.head()

[*********************100%%**********************]  1 of 1 completed


Date       Open       High        Low      Close  Adj Close   Volume
0 2023-01-03  87.385002  88.139999  84.300003  84.900002  84.900002  3901200
1 2023-01-04  85.889999  88.839996  84.790001  88.720001  88.720001  4817100
2 2023-01-05  88.480003  88.500000  86.750000  87.709999  87.709999  3419000
3 2023-01-06  88.519997  88.739998  85.709999  88.519997  88.519997  4348400
4 2023-01-09  90.254997  92.050003  88.879997  89.239998  89.239998  4319400

## Company News

In [20]:
finnhub_client = finnhub.Client(api_key="cnl0n1pr01qjvabn3kngcnl0n1pr01qjvabn3ko0")
news = pd.DataFrame(finnhub_client.company_news('ABNB', _from="2023-01-01", to='2024-01-01')).drop(columns=['id','image','category','url'])
news['datetime'] = pd.to_datetime(news['datetime'], unit='s').dt.strftime('%Y-%m-%d')
news['datetime'] = pd.to_datetime(news['datetime'])

news.head()

datetime  \
0 2024-01-01   
1 2024-01-01   
2 2024-01-01   
3 2024-01-01   
4 2024-01-01   

                                                                         headline  \
0          The Companies I Sold In 2023: Unveiling The Reasons Behind My Decision   
1                                      6 Top Long-Term Stocks On My 2024 Wishlist   
2  Is Airbnb (ABNB) Outperforming Other Computer and Technology Stocks This Year?   
3                                                             2023 Year In Review   
4                   Selling goods online? Be warned: HMRC will soon know about it   

  related        source  \
0    ABNB  SeekingAlpha   
1    ABNB  SeekingAlpha   
2    ABNB         Yahoo   
3    ABNB   TalkMarkets   
4    ABNB         Yahoo   

                                                                                                                                                         summary  
0  I focus on investing in the highest quality and growing businesses, while avoiding overpaying. Read about the positions I sold in 2023 and my goals for 2024.  
1       Discover six long-term investment ideas as the market soars in 2024, with some sectors presenting excellent value and potential for returns when they...  
2                                            Here is how Airbnb, Inc. (ABNB) and DoorDash, Inc. (DASH) have performed compared to their sector so far this year.  
3          Looking for stock market analysis and research with proves results? Zacks.com offers in-depth financial research with over 30years of proven results.  
4                                          Whatever it is, from used clothes on Vinted, collectables on eBay, to Airbnb lets, profits will now be under scrutiny